<a href="https://colab.research.google.com/github/jasreman8/OOPs-for-Intelligent-Agentic-Systems/blob/main/Decorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Objectives

*   Understand what Python decorators are and their basic syntax (`@decorator_name`).
*   Learn how decorators can modify or enhance the behavior of functions (or methods) without directly changing their original code.
*   See how to define a simple decorator.
*   Explore how LangChain utilizes decorators (specifically `@tool`) to easily transform regular Python functions into "Tools" usable by LangChain Agents.
*   Appreciate how combining Pydantic models (for input schema), type hints, and clear docstrings makes these decorated LangChain tools robust, self-documenting, and easier for LLMs to understand and use.


# Defining and Using Decorators: The Basics

Decorators are a form of metaprogramming in Python where you can add functionality to an existing function or method in a clean and readable way. Think of them as "wrapping" your function with another function that can do something before or after your original function runs, or even modify its behavior.

A decorator is essentially a function that takes another function as an argument (the decorated function) and returns a new function (usually a "wrapped" version of the original).

**Syntax:**
The `@decorator_name` syntax is syntactic sugar for applying a decorator.

In [1]:
def my_decorator(func_to_decorate):
    def wrapper_function(*args, **kwargs):
        # 1. Do something BEFORE calling the original function
        print(f"Wrapper: I am about to call {func_to_decorate.__name__}")
        #func_to_decorate.__name__ --> calls the name of the function which is passed in the decorator.

        # 2. Call the original function
        result = func_to_decorate(*args, **kwargs)
        # In this step the function is called using the *args for the function (in this case, it is Alice).

        # 3. Do something AFTER calling the original function
        print(f"Wrapper: I have called {func_to_decorate.__name__}")
        print(f"Wrapper: Original function returned: {result}")

        # 4. Return the result (or a modified result)
        return result
    return wrapper_function

## First Example

In [3]:
def say_hello(name: str):
    print(f"Original say_hello: Hello, {name}!")
    return f"Greetings to {name}"

greeting_output = say_hello("Alice")
greeting_output


Original say_hello: Hello, Alice!


'Greetings to Alice'

In [8]:
# Applying the decorator
@my_decorator
def say_hello(name: str):
    print(f"  Original say_hello: Hello, {name}!")
    return f"Greetings to {name}"

# --- Using the decorated functions ---
print("--- Calling say_hello with the decorator ---\n")
greeting_output = say_hello("Alice")
print(f"\nFinal output from say_hello: {greeting_output}\n")

--- Calling say_hello with the decorator ---

Wrapper: I am about to call say_hello
  Original say_hello: Hello, Alice!
Wrapper: I have called say_hello
Wrapper: Original function returned: Greetings to Alice

Final output from say_hello: Greetings to Alice



In [10]:
def add_numbers(a: int, b: int) -> int:
    print(f"Original add_numbers: Calculating {a} + {b}")
    return a + b

#When the function is run without the decorator
print("Running the function without the decorator.\n")
sum_output = add_numbers(5, 3)
sum_output

Running the function without the decorator.

Original add_numbers: Calculating 5 + 3


8

In [11]:
@my_decorator
def add_numbers(a: int, b: int) -> int:
    print(f"  Original add_numbers: Calculating {a} + {b}")
    return a + b

print("--- Calling add_numbers ---\n")
sum_output = add_numbers(5, 3)
print(f"\nFinal output from add_numbers: {sum_output}")

--- Calling add_numbers ---

Wrapper: I am about to call add_numbers
  Original add_numbers: Calculating 5 + 3
Wrapper: I have called add_numbers
Wrapper: Original function returned: 8

Final output from add_numbers: 8



**What happened?**
*   When you define `say_hello` with `@my_decorator` above it, Python essentially does this:
    `say_hello = my_decorator(say_hello)`
*   So, `say_hello` no longer refers to your original function directly. It now refers to `wrapper_function` returned by `my_decorator`.
*   When you call `say_hello("Alice")`, you're actually calling `wrapper_function("Alice")`.
*   `wrapper_function` then executes its pre-call logic, calls your *original* `say_hello` function (which `my_decorator` received as `func_to_decorate`), executes its post-call logic, and returns the result.

**Common Use Cases for Decorators:**
*   Logging function calls.
*   Timing function execution.
*   Enforcing access control or authentication.
*   Caching results.
*   Modifying arguments or return values.
*   **And in LangChain: Registering functions as Tools.**


Let us look at a practical example of decorators to register functions as Tools in LangChain.

# Creating Tools with Decorators in LangChain: The `@tool` Decorator

LangChain provides a convenient `@tool` decorator (from `langchain_core.tools`) that makes it incredibly easy to turn a regular Python function into a "Tool" that a LangChain Agent can use.

An Agent in LangChain is an LLM that can decide which "tools" to use to answer a question or accomplish a task. These tools are often functions that interact with the outside world (e.g., search the web, query a database, perform a calculation).

**Example: A Simple Business Calculator Tool**

Let's say we want to create a tool that calculates the final price of an item after applying a discount.


In [12]:
from langchain_core.tools import tool # The decorator
from pydantic import BaseModel, Field # For defining the input schema

In [13]:
# 1. Define the input schema for our tool using Pydantic
# This helps the LLM understand what arguments the tool expects.
class PriceCalculatorInput(BaseModel):
    original_price: float = Field(description="The original price of the item.")
    discount_percentage: float = Field(description="The discount percentage to apply (e.g., 10 for 10%). Must be between 0 and 100.")

In [14]:
# 2. Define the function and decorate it with @tool
@tool(args_schema=PriceCalculatorInput) # Pass the Pydantic model as args_schema
def calculate_discounted_price(original_price: float, discount_percentage: float) -> str:
    """
    Calculates the final price after applying a discount to an original price.
    Use this tool when you need to find the price of an item after a discount.
    For example, if an item is $100 and has a 15% discount.
    """
    if not (0 <= discount_percentage <= 100):
        return "Error: Discount percentage must be between 0 and 100."

    discount_amount = original_price * (discount_percentage / 100)
    final_price = original_price - discount_amount
    return f"The final price after a {discount_percentage}% discount on an original price of ${original_price:.2f} is ${final_price:.2f}."

In [15]:
type(calculate_discounted_price)

langchain_core.tools.structured.StructuredTool

Notice that the function is now a `Tool` object. This means that the wrapper `tool` decorator has transformed the function into a `Tool` object, affording it standard methods like `.invoke()`, `.name`, `.description`, `.args_schema` that the LangChain Agent framework can work with.

In [16]:
# --- How LangChain sees this tool ---
# The @tool decorator has transformed our function.
# Let's inspect some properties of the decorated tool object.
print("--- Tool Inspection ---\n")
print(f"Tool Name: {calculate_discounted_price.name}")
print(f"\nTool Description: {calculate_discounted_price.description}")
print(f"\nTool Args Schema: {calculate_discounted_price.args_schema.model_json_schema()}") # Shows the JSON schema

--- Tool Inspection ---

Tool Name: calculate_discounted_price

Tool Description: Calculates the final price after applying a discount to an original price.
Use this tool when you need to find the price of an item after a discount.
For example, if an item is $100 and has a 15% discount.

Tool Args Schema: {'properties': {'original_price': {'description': 'The original price of the item.', 'title': 'Original Price', 'type': 'number'}, 'discount_percentage': {'description': 'The discount percentage to apply (e.g., 10 for 10%). Must be between 0 and 100.', 'title': 'Discount Percentage', 'type': 'number'}}, 'required': ['original_price', 'discount_percentage'], 'title': 'PriceCalculatorInput', 'type': 'object'}


In [17]:
# We can also invoke it like a regular function (though an Agent would do this differently)
print("\n--- Manually Invoking the Decorated Function (as a test) ---\n")
result = calculate_discounted_price.invoke({"original_price": 150.00, "discount_percentage": 20.0})
print(result)


--- Manually Invoking the Decorated Function (as a test) ---

The final price after a 20.0% discount on an original price of $150.00 is $120.00.


In [18]:
result_invalid = calculate_discounted_price.invoke({"original_price": 100.00, "discount_percentage": 110.0})
print(result_invalid) # Pydantic validation (if used directly by Agent) or our manual check will catch this.

Error: Discount percentage must be between 0 and 100.


In [20]:
def simple_log(func):
    def wrapper(*args, **kwargs):
        print("Calling async func...")
        return func(*args, **kwargs)
    return wrapper

@simple_log
async def download():
    return "done"

# To run an async function in an environment where an event loop might already be running (like Colab),
# you should await it directly.
# If you need to ensure an event loop is available, you can check or use tools like `nest_asyncio`
# for more complex scenarios, but direct awaiting is often sufficient in interactive environments.

# Removed asyncio.run() and now directly awaiting the async function.
# This cell will now correctly execute the async function within the existing event loop.
# import asyncio

# # In a Colab environment, you can directly await the function.
# # If this were a standalone script, asyncio.run(download()) would be appropriate.
# result = await download()
# print(result)


**What does `@tool` do?**
1.  **Registers the Function:** It takes your `calculate_discounted_price` function.
2.  **Extracts Metadata:**
    *   **Name:** By default, it uses the function's name (`calculate_discounted_price`).
    *   **Description:** It automatically uses the function's **docstring** as the description of the tool. This is VERY important for the LLM Agent to understand what the tool does and when to use it!
    *   **Arguments Schema (`args_schema`):**
        *   If you provide `args_schema=PriceCalculatorInput` (our Pydantic model), the decorator uses this Pydantic model to define the expected input arguments, their types, and their descriptions. This information is crucial for the LLM Agent to know how to call the tool correctly.
        *   If you don't provide `args_schema`, it tries to infer it from the function's type hints. Using a Pydantic model is more robust and allows for better descriptions.
3.  **Creates a `Tool` Object:** It wraps your function in a `Tool` object (or a similar `StructuredTool` object) which has standard methods like `.invoke()`, `.name`, `.description`, `.args_schema` that the LangChain Agent framework can work with.

Now, `calculate_discounted_price` is not just a Python function; it's a LangChain `Tool` ready to be given to an Agent.